<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/new_face_recognition_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install deepface opencv-python pandas
!pip install streamlit deepface opencv-python scikit-learn numpy pillow

In [11]:
import cv2
from deepface import DeepFace
import numpy as np
import os
import pickle
from collections import defaultdict

class FaceRecognitionSystem:
    def __init__(self):
        self.known_faces = {}
        self.face_database = []
        self.threshold = 0.4
        self.model_name = "Facenet"
        self.detector_backend = "opencv"
        self.metrics = {
            'true_positives': 0,
            'false_positives': 0,
            'false_negatives': 0,
            'true_negatives': 0
        }
        self.confidence_history = []

    def load_known_faces(self, known_faces_dir):
        """Load known faces from a directory"""
        self.known_faces = {}
        for filename in os.listdir(known_faces_dir):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                name = os.path.splitext(filename)[0]
                img_path = os.path.join(known_faces_dir, filename)
                self.known_faces[name] = img_path

    def build_face_database(self):
        """Create embeddings for known faces"""
        self.face_database = []
        for name, img_path in self.known_faces.items():
            try:
                embedding = DeepFace.represent(
                    img_path=img_path,
                    model_name=self.model_name,
                    detector_backend=self.detector_backend
                )[0]["embedding"]
                self.face_database.append({"name": name, "embedding": embedding})
                print(f"Successfully added {name} to the database")
            except Exception as e:
                print(f"Error processing {name}: {str(e)}")

    def recognize_faces_in_image(self, image):
        """Recognize faces in a single image"""
        self.current_frame_metrics = defaultdict(int)
        recognized_faces = []

        try:
            # Convert image to numpy array if it's a file path
            if isinstance(image, str):
                img_array = cv2.imread(image)
                img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
            else:
                img_array = np.array(image)
                if len(img_array.shape) == 2:  # Convert grayscale to RGB
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_GRAY2RGB)
                else:
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

            detections = DeepFace.extract_faces(
                img_array,
                detector_backend=self.detector_backend,
                enforce_detection=False
            )

            for detection in detections:
                if detection["confidence"] > 0.9:
                    x, y, w, h = (
                        detection["facial_area"]["x"],
                        detection["facial_area"]["y"],
                        detection["facial_area"]["w"],
                        detection["facial_area"]["h"]
                    )

                    face_img = img_array[y:y+h, x:x+w]

                    try:
                        detected_embedding = DeepFace.represent(
                            img_path=face_img,
                            model_name=self.model_name,
                            enforce_detection=False
                        )

                        if detected_embedding:
                            detected_embedding = detected_embedding[0]["embedding"]
                            min_dist = float("inf")
                            identity = "Unknown"
                            confidence = 0

                            for person in self.face_database:
                                distance = DeepFace.distance(
                                    detected_embedding,
                                    person["embedding"],
                                    model_name=self.model_name
                                )
                                if distance < min_dist:
                                    min_dist = distance
                                    identity = person["name"]
                                    confidence = 1 - distance

                            if min_dist < self.threshold:
                                recognized_faces.append({
                                    "name": identity,
                                    "location": (x, y, w, h),
                                    "confidence": confidence
                                })
                                self.confidence_history.append(confidence)
                                self.current_frame_metrics['true_positives'] += 1
                            else:
                                self.current_frame_metrics['false_positives'] += 1
                    except Exception as e:
                        print(f"Recognition error: {str(e)}")
                        self.current_frame_metrics['false_positives'] += 1

            # Update metrics
            self.update_metrics()

            # Draw rectangles on the image
            for face in recognized_faces:
                x, y, w, h = face["location"]
                cv2.rectangle(img_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                label = f"{face['name']} ({face['confidence']:.2f})"
                cv2.putText(
                    img_array,
                    label,
                    (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (0, 255, 0),
                    2
                )

            return img_array, recognized_faces

        except Exception as e:
            print(f"Error in face detection: {str(e)}")
            return None, []

    def update_metrics(self):
        """Update performance metrics"""
        self.metrics['true_positives'] += self.current_frame_metrics['true_positives']
        self.metrics['false_positives'] += self.current_frame_metrics['false_positives']

    def calculate_metrics(self):
        """Calculate precision, recall, and F1-score"""
        precision = 0
        recall = 0
        f1 = 0

        total_positives = self.metrics['true_positives'] + self.metrics['false_negatives']
        if total_positives > 0:
            recall = self.metrics['true_positives'] / total_positives

        total_predicted_positives = self.metrics['true_positives'] + self.metrics['false_positives']
        if total_predicted_positives > 0:
            precision = self.metrics['true_positives'] / total_predicted_positives

        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)

        avg_confidence = np.mean(self.confidence_history) if self.confidence_history else 0

        return {
            "precision": precision,
            "recall": recall,
            "f1_score": f1,
            "average_confidence": avg_confidence,
            "true_positives": self.metrics['true_positives'],
            "false_positives": self.metrics['false_positives'],
            "false_negatives": self.metrics['false_negatives']
        }

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
